<a href="https://colab.research.google.com/github/abrahammathews2000/search4cyberplanet/blob/main/ML_model/ML_model_gcolab_v1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from PIL import Image
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [ ]:
# dir = '/content/drive/MyDrive/msAstroDataProject/'
x_dir  = '/content/drive/MyDrive/msAstroDataProject/lc_dict.npy'
x = np.load(x_dir)

y_dir  = '/content/drive/MyDrive/msAstroDataProject/shape_dict.npy'
y = np.load(y_dir)
y = y/255.0
